<a href="https://colab.research.google.com/github/the-fur-lover-sketch/effective-invention/blob/main/Music_player0921.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:



import numpy as np
from IPython.display import Audio
import ast
import operator as op

# ===============================
# 1️⃣ パラメータの定義
# ===============================
t_1 = 0.125
sp_1 = 0
f_1 = 220
f_s = 1
ds_1 = 0.7
di_1 = 0

sr = 44100     # サンプリング周波数

# 評価用変数マップ
vars_map = {
    "t_1": t_1,
    "sp_1": sp_1,
    "f_1": f_1,
    "f_s": f_s,
    "ds_1": ds_1,
    "di_1": di_1,
}

# ===============================
# 安全な式評価
# ===============================
_allowed_ops = {
    ast.Add: op.add,
    ast.Sub: op.sub,
    ast.Mult: op.mul,
    ast.Div: op.truediv,
    ast.Pow: op.pow,
    ast.USub: op.neg,
    ast.UAdd: lambda x: x,
    ast.FloorDiv: op.floordiv,
    ast.Mod: op.mod,
}

def safe_eval_expr(expr: str, names: dict):
    node = ast.parse(expr, mode="eval")
    def _eval(node):
        if isinstance(node, ast.Expression):
            return _eval(node.body)
        if isinstance(node, ast.Constant):
            return node.value
        if isinstance(node, ast.Num):
            return node.n
        if isinstance(node, ast.BinOp):
            left = _eval(node.left); right = _eval(node.right)
            op_type = type(node.op)
            if op_type in _allowed_ops:
                return _allowed_ops[op_type](left, right)
            raise ValueError(f"Unsupported op: {op_type}")
        if isinstance(node, ast.UnaryOp):
            operand = _eval(node.operand)
            op_type = type(node.op)
            if op_type in _allowed_ops:
                return _allowed_ops[op_type](operand)
            raise ValueError(f"Unsupported unary op: {op_type}")
        if isinstance(node, ast.Name):
            if node.id in names:
                return names[node.id]
            raise ValueError(f"Unknown name: {node.id}")
        if isinstance(node, (ast.List, ast.Tuple)):
            return [_eval(elt) for elt in node.elts]
        raise ValueError(f"Unsupported AST node: {type(node)}")
    return _eval(node.body)

# ===============================
# 2️⃣ GitHubから楽譜をダウンロード
# ===============================
!wget -O score.txt https://github.com/the-fur-lover-sketch/effective-invention/raw/refs/heads/main/Sheet1

# ===============================
# 3️⃣ テキスト楽譜を安全に読み込み
# ===============================
def parse_score_file(path, vars_map, default_waveform="sine"):
    score = []
    prev_start = prev_freq = prev_duration = None
    prev_waveform = default_waveform

    with open(path) as f:
        for raw in f:
            line = raw.strip()
            if not line or line.startswith("#"):
                continue
            parts = [p.strip() for p in line.split(",")]
            while len(parts) < 4:
                parts.append(".")
            start_field, freq_field, dur_field, wave_field = parts[:4]

            # start
            if start_field == ".":
                start = prev_start
            else:
                start = float(safe_eval_expr(start_field, vars_map))

            # freq
            if freq_field == ".":
                freq = prev_freq
            else:
                try:
                    freq_candidate = ast.literal_eval(freq_field)
                    if isinstance(freq_candidate, (int, float, list, tuple)):
                        freq = freq_candidate
                    else:
                        freq = safe_eval_expr(freq_field, vars_map)
                except Exception:
                    freq = safe_eval_expr(freq_field, vars_map)

            # duration
            if dur_field == ".":
                duration = prev_duration
            else:
                duration = float(safe_eval_expr(dur_field, vars_map))

            # waveform
            if wave_field == ".":
                waveform = prev_waveform
            else:
                try:
                    waveform = ast.literal_eval(wave_field)
                    if not isinstance(waveform, str):
                        waveform = str(waveform)
                except Exception:
                    waveform = wave_field.strip().strip('"').strip("'")

            # 整形
            if isinstance(freq, (list, tuple)):
                freq = [float(x) for x in freq]
            else:
                freq = float(freq)

            score.append((start, freq, duration, waveform))
            prev_start, prev_freq, prev_duration, prev_waveform = start, freq, duration, waveform

    return score

score = parse_score_file("score.txt", vars_map, default_waveform="sine")

# ===============================
# 4️⃣ 波形生成関数
# ===============================
def make_wave(freq, duration, waveform="sine"):
    t_arr = np.linspace(0, duration, int(sr*duration), False)
    if freq == 0:
        return np.zeros_like(t_arr)
    if waveform=="sine":
        return np.sin(2*np.pi*freq*t_arr)
    elif waveform=="square":
        return np.sign(np.sin(2*np.pi*freq*t_arr))
    elif waveform=="saw":
        return 2*(t_arr*freq - np.floor(0.5 + t_arr*freq))
    elif waveform=="triangle":
        return 2*np.arcsin(np.sin(2*np.pi*freq*t_arr))/np.pi
    else:
        raise ValueError("unknown waveform")

# ===============================
# 5️⃣ タイムラインに沿って合成
# ===============================
# total_duration をスコア末尾から自動計算
last_time = max((start + duration) for start, _, duration, _ in score)
total_duration = last_time + 0.5  # 末尾に少し余裕
music = np.zeros(int(sr * total_duration))

for start, f, duration, waveform in score:
    if isinstance(f, list):
        wave = sum(make_wave(fi, duration, waveform) for fi in f) / len(f)
    else:
        wave = make_wave(f, duration, waveform)
    start_i = int(start * sr)
    end_i = min(start_i + len(wave), len(music))
    music[start_i:end_i] += wave[:end_i-start_i]

# 正規化
music = music / np.max(np.abs(music))

# ===============================
# 6️⃣ 再生
# ===============================
Audio(music, rate=sr)


--2025-09-21 04:26:43--  https://github.com/the-fur-lover-sketch/effective-invention/raw/refs/heads/main/Sheet1
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/the-fur-lover-sketch/effective-invention/refs/heads/main/Sheet1 [following]
--2025-09-21 04:26:43--  https://raw.githubusercontent.com/the-fur-lover-sketch/effective-invention/refs/heads/main/Sheet1
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2953 (2.9K) [text/plain]
Saving to: ‘score.txt’

score.txt           100%[===================>]   2.88K  --.-KB/s    in 0s      

2025-09-21 04:26:44 (38.0 MB/s) - ‘score.txt’ saved [2953/2953]



/tmp/ipython-input-446859637.py:50: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):


In [3]:


from scipy.io.wavfile import write

# 音声データを16bit PCMに変換
write("music.wav", sr, (music * 32767).astype(np.int16))

from google.colab import files
files.download("music.wav")




<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>